<a href="https://colab.research.google.com/github/jain-m/whiz/blob/main/TravelAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
# prompt: set the environment OPENAI_API_KEY

!pip install openai
!pip install python-dotenv

import os
os.environ['OPENAI_API_KEY'] = "" #@param {type:"string"}
os.environ['WEATHER_API_KEY'] = "" #@param {type:"string"}


In [ ]:
#@title Chat completion app
# prompt: create a openai chat completion app

import os
from openai import OpenAI

# Avoid hardcoding API keys directly in your code.  Use environment variables instead.
# os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"  # Replace with your actual key or use environment variable

# Check if the API key is set
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

client = OpenAI()

def chat_completion(user_input):
    try:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo", # Using a more readily available model
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions."},
                {"role": "user", "content": user_input}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "An error occurred. Please try again."

# Example usage
user_input = "Tell me a joke."
response = chat_completion(user_input)
print(response)

# Interactive loop (optional)
while True:
  user_input = input("You: ")
  if user_input.lower() in ["exit", "quit"]:
    break
  response = chat_completion(user_input)
  print("Assistant:", response)


In [ ]:
# prompt: create an OpenAI assistant app with a code interpreter and a function. It should answer questions until user types quit or exit
# @title Chat completion app (updated)
import os
from openai import OpenAI
import json
import time
import requests

if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

assistant_id = "asst_qauDORPiRphvu3EfkHbXvEJr"

get_current_weather = { # query: What's the weather in London?
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},},
            "required": ["location"],},}

try:
  if assistant_id:
    print(f"Using existing assistant with ID: {assistant_id}")
    assistant = client.beta.assistants.retrieve(assistant_id)
  else:
    print("Creating a new assistant...")
    assistant = client.beta.assistants.create(
        name="MyMathTutor",
        instructions="You are a personal math tutor. Answer questions clearly and simply. Use the code interpreter when necessary.",
        model="gpt-3.5-turbo-1106",
        tools=[{"type": "code_interpreter"},
               {"type": "function", "function": get_current_weather}])
except client.error.OpenAIError as e:
    print(f"Error creating/retrieving assistant: {e}")
    assistant = None

thread = client.beta.threads.create()

def get_current_weather(location, unit="fahrenheit"):
    api_key = os.environ.get("WEATHER_API_KEY")  # Replace with your actual key or use environment variable
    if not api_key: raise ValueError("Please set the WEATHER_API_KEY environment variable.")

    try:
        base_url = "http://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": location,
            "appid": api_key,
            "units": "imperial" if unit == "fahrenheit" else "metric"
        }
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()
        temperature = data["main"]["temp"]
        description = data["weather"][0]["description"]
        return f"Current weather in {location}: {temperature} degrees {unit.lower()} and {description}."
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return f"Could not retrieve weather data for {location}. Please check your input or try again later."
    except KeyError as e:
        print(f"Unexpected response format from weather API: {e}")
        return f"Could not retrieve weather data for {location}.  The API returned an unexpected result."

def handle_actions(thread_id, run_id, tools):
    outputs = []
    for tool in tools:
      if tool.function.name == "get_current_weather":
        arguments = json.loads(tool.function.arguments)
        location = arguments.get("location")
        unit = arguments.get("unit", "fahrenheit") # Provide default unit
        response = get_current_weather(location, unit)
        # response = f"Simulated weather for {location}: 70 degrees Fahrenheit and sunny."
        outputs.append({"tool_call_id": tool.id, "output": response,})
    client.beta.threads.runs.submit_tool_outputs(thread_id=thread_id, run_id=run_id, tool_outputs=outputs)

def chat_completion(assistant_id, thread_id, user_input):
    try:
        response = client.beta.threads.messages.create(thread_id=thread_id, role="user", content=user_input)
        run = client.beta.threads.runs.create(thread_id=thread_id, assistant_id=assistant_id,)

        # Wait for the run to complete
        while True:
          run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
          if run_status.status == "requires_action":
              handle_actions(thread_id, run.id, run_status.required_action.submit_tool_outputs.tool_calls)
          elif run_status.status in ["completed", "failed", "cancelled", "expired"]:
              run_status.status != "completed" and print(f"Run finished with status: {run_status.status}")
              break;
          else:
              time.sleep(1)

        # Return the assistant's message
        for msg in client.beta.threads.messages.list(thread_id=thread_id).data:
          if msg.role == "assistant": return msg.content[0].text.value
    except Exception as e:
        print(f"An error occurred: {e}")
        return "An error occurred. Please try again."

while True:
  user_input = input("You: ")
  if user_input.lower() in ["exit", "quit"]:
    break
  response = chat_completion(assistant.id, thread.id, user_input)
  print("Assistant:", response)

response = client.beta.threads.delete(thread.id)
print(response)


In [ ]:
# prompt: Write a function to handle ETA queries using Google maps
#@title Driving ETA via Maps API

def get_eta(origin, destination, mode_of_transport="driving"):
    """
    Fetches the estimated time of arrival (ETA) between two locations using the Google Maps Distance Matrix API.

    Args:
        origin: The starting location (e.g., "New York City, NY" or latitude/longitude coordinates).
        destination: The ending location (e.g., "Los Angeles, CA" or latitude/longitude coordinates).
        mode_of_transport: The mode of transportation (e.g., "driving", "walking", "bicycling", "transit").

    Returns:
        A string representing the ETA, or an error message if the API call fails.
    """
    try:
        # Replace 'YOUR_API_KEY' with your actual Google Maps API key
        api_key = "YOUR_API_KEY"
        url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin}&destinations={destination}&mode={mode_of_transport}&key={api_key}"

        response = requests.get(url)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        if data['status'] == 'OK':
          duration = data['rows'][0]['elements'][0]['duration']['text']
          return f"The estimated travel time from {origin} to {destination} is {duration}."
        else:
            return f"Error: {data['status']}"

    except requests.exceptions.RequestException as e:
        return f"An error occurred during the API request: {e}"
    except (KeyError, IndexError) as e:
        return f"Error: Unexpected response format from API. {e}"


#Example Usage
origin = "New York City, NY"
destination = "Los Angeles, CA"
eta = get_eta(origin, destination)
eta
